### multiple subdomain 결과 재정렬

In [ ]:
# import json
# domain = '경제'
# with open(f'evaluation/eval_data/multiple_subdomain_results/{domain}_subdomain_classified.json', 'r', encoding='utf-8') as f:
    # data = json.load(f)

#  new_data = []
# for d in data:
#     if d['qna_subdomain_reason'] in d['qna_reason']:
#         d['qna_reason'] = d['qna_reason'].replace(d['qna_subdomain_reason'], '').strip()
#         # print(d)
#     new_d = {
#         'file_id': d['file_id'],
#         'title': d['title'],
#         'chapter': d['chapter'],
#         'qna_id': d['qna_id'],
#         'qna_domain': d['qna_domain'],
#         'qna_subdomain': d['qna_subdomain'],
#         'qna_domain_reason': d['qna_reason'],
#         'qna_subdomain_reason': d['qna_subdomain_reason'],
#         'qna_question': d['qna_question'],
#         'qna_options': d['qna_options'],
#         'qna_answer': d['qna_answer'],
#         'qna_explanation': d['qna_explanation']
#     }
#     new_data.append(new_d)
    
# with open(f'evaluation/eval_data/subdomain_results_old/{domain}_subdomain_classified.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

## 모의고사 만들기 파이프라인

In [ ]:
import os, json, random
from datetime import datetime

random.seed(42)

# 과목 분류
BBASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
with open(os.path.join(BBASE_DIR, 'exam_hierarchy.json'), 'r', encoding='utf-8') as f:
    subjects = json.load(f)

# subdomain 분류 완료된 데이터 > 모의고사 대상 문제주머니 추출
BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

with open(os.path.join(BASE_DIR, 'classification_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

# 로그 파일 설정
log_output = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_pipeline_{timestamp}.log'

for json_file in os.listdir(BASE_DIR):
    if not json_file.endswith("_subdomain_classified.json"):
        continue
    
    with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    domain = json_file.replace("_subdomain_classified.json", "")
    log_message = f"=====================================\n<<< {domain} 분야 모의고사용 문제 추출 >>>"
    print(log_message)
    log_output.append(log_message)
    
    total_questions = len(data)
    if total_questions == stats[domain]['total_questions']:
        mock_data = []
        need_more = {}
        mock_total_questions = round(total_questions * 0.3)
        
        log_message1 = f"원본 문제 수: {total_questions}"
        log_message2 = f"추출 대상 문제 수: {mock_total_questions}"
        print(log_message1)
        print(log_message2)
        log_output.extend([log_message1, log_message2])

        # subdomain 별로 모의고사 만들기
        for subdomain in stats[domain]['subdomain_distribution'].keys():
            subdomain_data = [d for d in data if d['qna_subdomain'] == subdomain]
            random.shuffle(subdomain_data)
            try:
                mock_subdomain_data = random.sample(subdomain_data, mock_total_questions // len(stats[domain]['subdomain_distribution']))
            except ValueError:
                mock_subdomain_data = subdomain_data
                need_more[subdomain] = mock_total_questions // len(stats[domain]['subdomain_distribution']) - len(mock_subdomain_data)
            
            log_message = f"  - {subdomain}: {len(mock_subdomain_data)}"
            print(log_message)
            log_output.append(log_message)
            mock_data.extend(mock_subdomain_data)

        log_message1 = f" ====> 총 추출 문제: {len(mock_data)}"
        print(log_message1)
        log_output.append(log_message1)
        
        if need_more:
            log_message2 = f" ====> 모자란 문제: {need_more}"
            print(log_message2)
            log_output.append(log_message2)
    else:
        log_message = "갯수가 안맞아!"
        print(log_message)
        log_output.append(log_message)
    
    # 문제주머니 저장
    for domains in subjects.values():
        if domain in domains:
            subject = list(subjects.keys())[list(subjects.values()).index(domains)]
    
    OUTPUT_DIR = f"{BBASE_DIR}/mock_data/{subject}"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(f'{OUTPUT_DIR}/{domain}_mock_data.json', 'w', encoding='utf-8') as f:
        json.dump(mock_data, f, ensure_ascii=False, indent=4)

# 로그 파일 저장
os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
with open(log_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(log_output))

print(f"\n로그가 저장되었습니다: {log_file}")

### 수정한 도메인 원본에 반영하기?

### 분류 조정하기

In [ ]:
# import os, json

# BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/multiple_with_subdomain'

# for json_file in os.listdir(BASE_DIR):
#     if not json_file.endswith("_subdomain_classified.json"):
#         continue
    
#     with open(os.path.join(BASE_DIR, json_file), 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     domain = json_file.replace("_subdomain_classified.json", "")
    
#     new_data = []
#     other_domain = {}
#     for d in data:
#         if d['qna_domain'] != domain:
#             other_domain[d['qna_domain']] = d
#         else:
#             new_data.append(d)
    
#     with open(os.path.join(BASE_DIR, f'{domain}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#         json.dump(new_data, f, ensure_ascii=False, indent=4)
        
#     for other_domain_key in other_domain.keys():
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'r', encoding='utf-8') as f:
#             other_domain_data = json.load(f)
        
#         other_domain_data.append(other_domain[other_domain_key])
                
#         with open(os.path.join(BASE_DIR, f'{other_domain_key}_subdomain_classified.json'), 'w', encoding='utf-8') as f:
#             json.dump(other_domain_data, f, ensure_ascii=False, indent=4)

### mock 데이터에서 모의고사 문제 추출

In [ ]:
import os, json, random
from datetime import datetime

BBASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data'
BASE_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/mock_data'
OUTPUT_DIR = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/evaluation/eval_data/mock_exam'

with open(os.path.join(BBASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

# 로그 파일 설정
log_output = []
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
log_file = f'/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs/mock_exam_extraction_{timestamp}.log'

for subject in os.listdir(BASE_DIR):
    if subject.endswith(".json"):
        continue
    
    log_message = f"=====================================\n {subject}"
    print(log_message)
    log_output.append(log_message)
    
    mock_exam_data = []
    exam_questions = 250
    for json_file in os.listdir(os.path.join(BASE_DIR, subject)):
        if not json_file.endswith("_mock_data.json"):
            continue
        
        with open(os.path.join(BASE_DIR, subject, json_file), 'r', encoding='utf-8') as f:
            json_file = json.load(f)
        
        domain = os.path.basename(f.name).replace("_mock_data.json", "")
        if domain == '디지털':
            continue
        log_message = f"-------------------------------------\n {domain}"
        print(log_message)
        log_output.append(log_message)
    
        # subdomain 별로 모의고사 만들기
        for subdomain in stats[subject][domain]['exam_subdomain_distribution'].keys():
            subdomain_data = [d for d in json_file if d['qna_subdomain'] == subdomain]
            random.shuffle(subdomain_data)
            try:
                random.seed(42)
                mock_subdomain_data = random.sample(subdomain_data, stats[subject][domain]['exam_subdomain_distribution'][subdomain])
                log_message = f" - {subdomain}: {len(mock_subdomain_data)}"
                print(log_message)
                log_output.append(log_message)
            except ValueError:
                mock_subdomain_data = subdomain_data
                log_message = f" - (ERROR) {subdomain}: {len(mock_subdomain_data)}/{stats[subject][domain]['exam_subdomain_distribution'][subdomain]}"
                print(log_message)
                log_output.append(log_message)
                
            mock_exam_data.extend(mock_subdomain_data)
        
    log_message = f" ====> 총 추출 문제: {len(mock_exam_data)}/{exam_questions} ({len(mock_exam_data)/exam_questions:.2%})"
    print(log_message)
    log_output.append(log_message)
    
    # 출력 디렉토리 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    with open(f'{OUTPUT_DIR}/{subject}_mock_exam.json', 'w', encoding='utf-8') as f:
        json.dump(mock_exam_data, f, ensure_ascii=False, indent=4)

# 로그 파일 저장
os.makedirs('/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/logs', exist_ok=True)
with open(log_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(log_output))

print(f"\n로그가 저장되었습니다: {log_file}")

### multiple options 오류들

In [ ]:
import json
import re

with open(os.path.join(BBASE_DIR, 'exam_statistics.json'), 'r', encoding='utf-8') as f:
    stats = json.load(f)

for subject in stats.keys():
    print("================\n", subject)
    with open(f'evaluation/eval_data/mock_exam/{subject}_mock_exam.json', 'r', encoding='utf-8') as f:
        data = json.load(f)

    # ①②③④⑤로 시작하지 않는 경우를 찾기
    invalid_options = []

    for d in data:
        if d['qna_options'] and isinstance(d['qna_options'], list):
            options = d['qna_options']
            
            # 각 옵션을 확인
            for i, option in enumerate(options):
                # ①②③④⑤로 시작하지 않는 경우 찾기
                if not re.match(r'^[①②③④⑤]', option.strip()):
                    invalid_options.append({
                        'file_id': d['file_id'],
                        'qna_domain': d['qna_domain'],
                        'qna_subdomain': d['qna_subdomain'],
                        'qna_id': d['qna_id'],
                        'option_index': i,
                        'original_option': option,
                        # 'first_char': option.strip()[0] if option.strip() else ''
                    })

    print(f"①②③④⑤로 시작하지 않는 options 수: {len(invalid_options)}")
    print(invalid_options)

json_rev 자리 빌리기

In [3]:
import pandas as pd
import os

base_path = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 3

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3, engine='openpyxl')[['관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
lv
# print(lv == 'Lv2')

In [4]:
# Fixed version - using Python file operations instead of sed
import shutil
from tqdm import tqdm
import fitz

dpi = 300
mat = fitz.Matrix(dpi / 150, dpi / 150)

# for tn, isbn, lv in tqdm(zip(excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
for tn in tqdm(excel_analy['관리번호']):
   # if '/' in lv: 
   #    lv = lv.replace("/", '_')
   
   # if 'Lv' not in lv:
   #    continue
   lv = excel_analy[excel_analy['관리번호'] == tn]['분류'].tolist()[0]
   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2 && mkdir {tn} && mv {tn}*.json {tn}/ && cd {tn} && mkdir pdf_page")
      # pdf_page 만들기
      pdf_path = f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv1/{tn}.pdf"
      doc = fitz.open(pdf_path)
      for p, page in enumerate(doc):
         img = page.get_pixmap(matrix=mat)
         img.save(f"/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C/Lv2/{tn}/pdf_page/{tn}_{p+1:04d}.png")
      
      # break

   # SS0041 - SS0045 사이에서 MuPDF error: syntax error: invalid key in dict

100%|██████████| 54/54 [02:54<00:00,  3.24s/it]


In [ ]:
img